# aprior analysis

### after clustering,it will generate k classes,each class will have a list of user_ids inside that class,next step is to apply basket anaysis on each classification,to get the rules of each class

In [1]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')

# fresh fruits:24
# fresh vegetables:83
# packaged vegetables fruits:122

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def Apriori(user_product_part1):
    
    user_product_dic={}
    for item in user_product_part1.itertuples():
        if(user_product_dic.get(item[2])):
            user_product_dic[item[2]].append(item[3])
        else:
            user_product_dic[item[2]] = [] 
            user_product_dic[item[2]].append(item[3])
    transaction = []
    for item in user_product_dic:
        transaction.append(user_product_dic[item])
#     print('transaction',transaction)
    te = TransactionEncoder()
    te_ary = te.fit(transaction).transform(transaction)
#     print('te_ary',te_ary)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    return df

In [3]:
def transform_associationRule_to_product(association_rule):
    
    relationship_rule = association_rule[['antecedents','consequents']]
    arr_relation=relationship_rule.values
    final_rules = []
    for item in arr_relation:
        pre_product_Arr = []
        after_product_Arr = []
        pre = item[0]
        after = item[1]
        for pre_item in pre:
            pre_product_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
        for after_item in after:
            after_product_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
        tup = (pre_product_Arr,'----->',after_product_Arr)
        final_rules.append(tup)
    return final_rules

In [4]:
def transform_frequentItem_to_product(frequent_itemsets):
    arr_relation=frequent_itemsets.values
    final_arr = []
    for item in arr_relation:
        arr = []
        for it in item[1]:
            arr.append(products[products['product_id' ]== it]['product_name'].values[0])
        final_arr.append(arr)
    return final_arr

#  association items of all users (no any change)

In [5]:
order_user_id = orders[['order_id','user_id']]

order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
# order_products_id = order_products_id2
order_products_id
user_product_id=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')
user_product_id

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [6]:
df = Apriori(user_product_id)
frequent_itemsets_all = apriori(df, min_support=0.1, use_colnames=True)
association_rule_all = association_rules(frequent_itemsets_all, metric="confidence", min_threshold=0.1)

In [7]:
transform_associationRule_to_product(association_rule_all)

[(['Bag of Organic Bananas'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Banana']),
 (['Banana'], '----->', ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], '----->', ['Organic Hass Avocado']),
 (['Organic Hass Avocado'], '----->', ['Bag of Organic Bananas']),
 (['Banana'], '----->', ['Strawberries']),
 (['Strawberries'], '----->', ['Banana']),
 (['Organic Strawberries'], '----->', ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Banana']),
 (['Banana'], '----->', ['Organic Strawberries']),
 (['Limes'], '----->', ['Organic Strawberries']),
 (['Organic Strawberries'], '----->', ['Limes']),
 (['Organic Strawberries'], '----->', ['Organic Blueberries']),
 (['Organi

In [8]:
s = frequent_itemsets_all.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)


[['Banana'],
 ['Bag of Organic Bananas'],
 ['Organic Strawberries'],
 ['Organic Baby Spinach'],
 ['Large Lemon'],
 ['Limes'],
 ['Strawberries'],
 ['Organic Hass Avocado'],
 ['Organic Avocado'],
 ['Organic Blueberries'],
 ['Organic Garlic'],
 ['Organic Yellow Onion'],
 ['Organic Zucchini'],
 ['Organic Raspberries'],
 ['Bag of Organic Bananas', 'Organic Strawberries'],
 ['Yellow Onions'],
 ['Cucumber Kirby'],
 ['Organic Grape Tomatoes'],
 ['Organic Strawberries', 'Organic Baby Spinach'],
 ['Seedless Red Grapes']]

#  association items of all users (remove asiles : fresh fruits(24),fresh vegetables(83),packaged vegetables fruits(122))

### find the top 3 aisles [fresh fruits,fresh vegetables,packaged vegetables fruits]

In [9]:
aisle_ids = []
aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for item in aisle_arr:
    aisle_ids.append(item[0])

product_ids = []
product_arr = products[products['aisle_id'].isin(aisle_ids)].values
for item in product_arr:
    product_ids.append(item[0])
product_ids
aisle_ids

[24, 83, 123]

In [10]:
user_product_id = user_product_id[~user_product_id['product_id'].isin(product_ids)]

In [11]:
df = Apriori(user_product_id)
t1 = time.time()
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

time t2 - t1 17.396481037139893


In [12]:
association_rule = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
# association_rule

In [13]:
transform_associationRule_to_product(association_rule)

[]

In [14]:
s = frequent_itemsets.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Blueberries'],
 ['Organic Whole Milk']]

# after removing frequent asiles and apply association rule on each user classification 

In [15]:
user_classification = pd.read_csv('../customer_classificationV2.csv')
user_classification = user_classification[['user_id','label']]
user0 = user_classification[user_classification['label']==0]
user1 = user_classification[user_classification['label']==1]
user2 = user_classification[user_classification['label']==2]
user3 = user_classification[user_classification['label']==3]
user4 = user_classification[user_classification['label']==4]

#  association items of clustering 0

In [17]:
user_product_part0 = user_product_id[user_product_id['user_id'].isin(user0['user_id']) ]
df = Apriori(user_product_part0)
t1 = time.time()
frequent_itemsets0 = apriori(df, min_support=0.1, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=0.1)
# association_rule0

time t2 - t1 2.518946647644043


### top association rules

In [18]:
transform_associationRule_to_product(association_rule0)

[(['Extra Virgin Olive Oil'], '----->', ['Organic Cilantro']),
 (['Organic Cilantro'], '----->', ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Extra Virgin Olive Oil']),
 (['Organic Cilantro'], '----->', ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], '----->', ['Organic Cilantro'])]

### top 20 support value

In [19]:
s = frequent_itemsets0.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Organic Whole Milk'],
 ['Blueberries'],
 ['Bunched Cilantro'],
 ['Grated Parmesan'],
 ['Original Hummus'],
 ['Unsalted Butter'],
 ['Half & Half'],
 ['Organic Black Beans'],
 ['Organic Basil'],
 ['Boneless Skinless Chicken Breasts'],
 ['Organic Half & Half'],
 ['Feta Cheese Crumbles'],
 ['Organic Garbanzo Beans'],
 ['Organic Coconut Milk'],
 ['Organic Cilantro', 'Organic Italian Parsley Bunch'],
 ['Organic Thyme'],
 ['Organic Sour Cream']]

#  association items of clustering 1

In [20]:
user_product_part1 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]
df = Apriori(user_product_part1)
t1 = time.time()
frequent_itemsets1 = apriori(df, min_support=0.2, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

association_rule1 = association_rules(frequent_itemsets1, metric="confidence", min_threshold=0.2)
# association_rule1

time t2 - t1 0.07628583908081055


### top association rules

In [21]:
transform_associationRule_to_product(association_rule1)

[(['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Organic Whole String Cheese'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Organic Whole String Cheese']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  '----->',
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot'])]

### top 20 support value

In [22]:
s = frequent_itemsets1.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Whole Milk'],
 ['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
 ['Spinach Peas & Pear Stage 2 Baby Food'],
 ['Organic Whole String Cheese'],
 ['Grated Parmesan'],
 ['Organic Italian Parsley Bunch'],
 ['Broccoli & Apple Stage 2 Baby Food'],
 ['Extra Virgin Olive Oil'],
 ['Organic Cilantro'],
 ['Original Hummus'],
 ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
 ['Organic Pears, Peas and Broccoli Puree Stage 1'],
 ['Peach,  Apricot & Banana Stage 2 Baby Food'],
 ['Blueberries'],
 ['Happy Baby Spinach, Mango, and Pear Baby Food'],
 ['Organic Mixed Berry Yogurt & Fruit Snack'],
 ['Organic Black Beans'],
 ['Sweeet Potato, Apple, Carrot & Cinnamon Organic Superfoods'],
 ['Baby Food Stage 2 Blueberry Pear & Purple Carrot',
  'Spinach Peas & Pear Stage 2 Baby Food'],
 ['Baby Food Stage 2 Pumpkin Banana']]

#  association items of clustering 2

In [23]:
user_product_part1 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]
df = Apriori(user_product_part1)
t1 = time.time()
frequent_itemsets2 = apriori(df, min_support=0.1, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

association_rule2 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.1)
# association_rule2

time t2 - t1 0.42274904251098633


### top association rules

In [24]:
transform_associationRule_to_product(association_rule2)

[(['Organic Diced Tomatoes'], '----->', ['Organic Whole Milk']),
 (['Organic Whole Milk'], '----->', ['Organic Diced Tomatoes']),
 (['Organic Whole Grain Oatmeal Cereal Baby Food'],
  '----->',
  ['Organic Whole Milk']),
 (['Organic Whole Milk'],
  '----->',
  ['Organic Whole Grain Oatmeal Cereal Baby Food']),
 (['Peach,  Apricot & Banana Stage 2 Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Peach,  Apricot & Banana Stage 2 Baby Food']),
 (['Baby Food Stage 2 Pumpkin Banana'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Baby Food Stage 2 Pumpkin Banana']),
 (['Happy Baby Spinach, Mango, and Pear Baby Food'],
  '----->',
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  '----->',
  ['Happy Baby Spinach, Mango, and Pear Baby Food']),
 (['Organic Whole String Cheese'],
  '----->',
  ['Broccoli & Apple Stag

### top 20 support value

In [25]:
s = frequent_itemsets2.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Whole Milk'],
 ['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
 ['Spinach Peas & Pear Stage 2 Baby Food'],
 ['Organic Whole String Cheese'],
 ['Grated Parmesan'],
 ['Organic Italian Parsley Bunch'],
 ['Broccoli & Apple Stage 2 Baby Food'],
 ['Extra Virgin Olive Oil'],
 ['Organic Cilantro'],
 ['Original Hummus'],
 ['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'],
 ['Organic Pears, Peas and Broccoli Puree Stage 1'],
 ['Peach,  Apricot & Banana Stage 2 Baby Food'],
 ['Happy Baby Spinach, Mango, and Pear Baby Food'],
 ['Blueberries'],
 ['Organic Mixed Berry Yogurt & Fruit Snack'],
 ['Organic Black Beans'],
 ['Sweeet Potato, Apple, Carrot & Cinnamon Organic Superfoods'],
 ['Baby Food Stage 2 Blueberry Pear & Purple Carrot',
  'Spinach Peas & Pear Stage 2 Baby Food'],
 ['Baby Food Stage 2 Pumpkin Banana']]

#  association items of clustering 3

In [26]:
user_product_part3 = user_product_id[user_product_id['user_id'].isin(user3['user_id']) ]
df = Apriori(user_product_part3)
t1 = time.time()
frequent_itemsets3 = apriori(df, min_support=0.3, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

association_rule3 = association_rules(frequent_itemsets3, metric="confidence", min_threshold=0.3)
# association_rule3

time t2 - t1 0.012366056442260742


### top association rules

In [27]:
transform_associationRule_to_product(association_rule3)

[(['Packaged Grape Tomatoes'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Packaged Grape Tomatoes']),
 (['Clementines'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Clementines']),
 (['Clementines'], '----->', ['Raspberries']),
 (['Raspberries'], '----->', ['Clementines']),
 (['Packaged Grape Tomatoes'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Packaged Grape Tomatoes']),
 (['Packaged Grape Tomatoes'], '----->', ['Raspberries']),
 (['Raspberries'], '----->', ['Packaged Grape Tomatoes']),
 (['Rainbow Bell Peppers'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Rainbow Bell Peppers']),
 (['Seedless Cucumbers'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Seedless Cucumbers']),
 (['Raspberries'], '----->', ['Hass Avocados']),
 (['Hass Avocados'], '----->', ['Raspberries'])]

### top 20 support value

In [28]:
s = frequent_itemsets3.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Hass Avocados'],
 ['Clementines'],
 ['Raspberries'],
 ['Packaged Grape Tomatoes'],
 ['Seedless Cucumbers'],
 ['Clementines', 'Hass Avocados'],
 ['Raspberries', 'Hass Avocados'],
 ['Baby Cucumbers'],
 ['Clementines', 'Raspberries'],
 ['Rainbow Bell Peppers'],
 ['Packaged Grape Tomatoes', 'Hass Avocados'],
 ['Seedless Cucumbers', 'Hass Avocados'],
 ['Packaged Grape Tomatoes', 'Clementines'],
 ['Blueberries'],
 ['Packaged Grape Tomatoes', 'Raspberries'],
 ['Green Seedless Grapes'],
 ['Rainbow Bell Peppers', 'Hass Avocados'],
 ['Organic Tortilla Chips']]

#  association items of clustering 4

In [29]:
user_product_part4 = user_product_id[user_product_id['user_id'].isin(user4['user_id']) ]
df = Apriori(user_product_part4)
t1 = time.time()
frequent_itemsets4 = apriori(df, min_support=0.15, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

association_rule4 = association_rules(frequent_itemsets4, metric="confidence", min_threshold=0.15)
# association_rule4

time t2 - t1 0.2188570499420166


### top association rules

In [30]:
transform_associationRule_to_product(association_rule4)

[(['Sparkling Lemon Water'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Water Grapefruit'], '----->', ['Sparkling Lemon Water']),
 (['Sparkling Lemon Water'], '----->', ['Sparkling Water Grapefruit']),
 (['Sparkling Water Grapefruit'],
  '----->',
  ['Peach Pear Flavored Sparkling Water']),
 (['Peach Pear Flavored Sparkling Water'],
  '----->',
  ['Sparkling Water Grapefruit']),
 (['Sparkling Water Grapefruit'], '----->', ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], '----->', ['Sparkling Water Grapefruit'])]

### top 20 support value

In [31]:
s = frequent_itemsets4.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Sparkling Water Grapefruit'],
 ['Lime Sparkling Water'],
 ['Extra Virgin Olive Oil'],
 ['Sparkling Lemon Water'],
 ['Sparkling Water Grapefruit', 'Lime Sparkling Water'],
 ['Organic Half & Half'],
 ['Sparkling Natural Mineral Water'],
 ['Organic Italian Parsley Bunch'],
 ['Organic Cilantro'],
 ['Half & Half'],
 ['Original Hummus'],
 ['Spring Water'],
 ['Blueberries'],
 ['Peach Pear Flavored Sparkling Water'],
 ['Sparkling Water Grapefruit', 'Sparkling Lemon Water'],
 ['Sparkling Lemon Water', 'Lime Sparkling Water'],
 ['Sparkling Water Berry'],
 ['Organic Whole Milk'],
 ['100% Recycled Paper Towels'],
 ['Sparkling Water Grapefruit', 'Peach Pear Flavored Sparkling Water']]